In [47]:
import numpy as np

def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ])
        

def normalize_quaternion(q):
    norm = np.linalg.norm(q)
    if norm == 0:
        return q
    return q / norm


def update_orientation(q, omega, dt):
    omega = rotate_vector(omega,q)
    theta = np.linalg.norm(omega)
    sin_half_theta = np.sin(theta*dt/2)
    dq = np.array([np.cos(theta*dt/2), sin_half_theta*omega[0]/theta, sin_half_theta*omega[1]/theta, sin_half_theta*omega[2]/theta])
    dq = normalize_quaternion(dq)
    q = quaternion_multiply(dq,q)
    q = normalize_quaternion(q)
    return dq, q

        
def rotate_vector(v, q):
    v_quat = np.array([0] + list(v))  # Create a quaternion with zero scalar part
    q_conj = np.array([q[0], -q[1], -q[2], -q[3]])  # Conjugate of q
    v_rotated_quat = quaternion_multiply(quaternion_multiply(q, v_quat), (q_conj))
    return v_rotated_quat[1:]  # Extract the vector part


i = 0
size = 1000
dt = 1/size
n = np.array([0,0,1])

for i in range(size):
    angular_speed = np.array([0,np.pi/2,0])
    if i == 0:
        q = np.array([0,0,0,1])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))


print(n)





[-1.00000000e+00  0.00000000e+00  2.16276649e-15]


In [43]:
print('....................................................................................')
i = 0
for i in range(size):
    angular_speed = np.array([0,np.pi/2,0])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))
print(n)
i = 0
for i in range(size):
    angular_speed = np.array([np.pi,0,0])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))
print(n)

for i in range(size):
    angular_speed = np.array([0, np.pi,0])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))
print(n)

i = 0
for i in range(size):
    angular_speed = np.array([0,0,np.pi])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))
print(n)

i = 0
for i in range(size):
    angular_speed = np.array([np.pi/2,0,0])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))
print(n)
i = 0
for i in range(size):
    angular_speed = np.array([0,-np.pi/4,0])
    i += 1
    dq, q = update_orientation(q, angular_speed, dt)
    n = normalize_quaternion(rotate_vector(n, dq))
print(n)

[-1.00000000e+00  2.23871866e-15  7.83034738e-14]
[-7.65448902e-14 -3.38964640e-14 -1.00000000e+00]
[-5.86024712e-14  7.07106781e-01 -7.07106781e-01]
